In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
def rotate_image(x):
    return x.rotate(90)

In [2]:
pip install efficientnet_pytorch

In [3]:
import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset,DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet
from torchvision.transforms import InterpolationMode

In [4]:
save_path = '/content/scalp_weights/'

In [5]:
def save_checkpoint(epoch, model, optimizer, acc):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': acc,
    }
    torch.save(checkpoint, save_path + 'checkpoint.pth')

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
#배치 사이즈 조절
hyper_param_batch = 32
#랜덤성을 한가지로 맞추기 위해 시드 입력
random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

num_classes = 4 # 분류할 클래스가 4개라는 뜻.
model_name = 'efficientnet-b7' # 모델명
# 폴더 경로 정하기
train_name = 'model2'

PATH = '/content/scalp_weights/'

data_train_path = '/content/gdrive/MyDrive/딥러닝/두피/비듬 img/'
data_validation_path = '/content/gdrive/MyDrive/딥러닝/두피/비듬 vail img/'

image_size = EfficientNet.get_image_size(model_name)
print(image_size)
# 모델 이름은 Efficientnet , 클래스는 4개로 분류하는 모델 생성
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes)
checkpoint = torch.load(PATH + 'checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device) # 모델을 GPU(없으면 cpu)에 적용
# 데이터의 증강 등 정제하는 과정
#interpolation :
# NEAREST(0)(얘가 제일 빠름) : 가장 가까운 이웃 픽셀의 값을 그대로 사용
# BOX(4) : 주변 픽셀의 가중평균을 사용
# BILINEAR(2) : 가까운 4개의 픽셀을 사용하여 선형 보간을 수행 , NEAREST보다 부드러움
# LANCZOS(1)(얘가 제일 느림) , BICUBIC(3)은 각각 BILINEAR의 기반이지만 , 더 정교하게 보간 , BILINEAR -> BICUBIC -> LANCZOS 순으로 느림
# HAMMING(5) : BOX와 유사하지만 , 얘가 더 정교하고 느림
# 가중 평균 : 주변 픽셀에 가중치를 곱해서 각각 더하고 , 줬던 가중치의 총합으로 나눠서 나오는 값이 가중평균
# 선형 보간 : 선으로 잇고 , 그 영역 안에서의 보간을 수행하는 것 ( 직선이면 직선상에서 )
transforms_train = transforms.Compose([
                                        transforms.Resize([int(300), int(300)], interpolation = InterpolationMode.BILINEAR),#크기 조절 , interpolation은 어떤 형식으로 보간할건지 정하는 것.
                                        transforms.RandomHorizontalFlip(p=0.5),
                                        transforms.RandomVerticalFlip(p=0.5),
                                        transforms.Lambda(lambda x: x.rotate(90)),
                                        transforms.RandomRotation(10),
                                        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                                        transforms.ToTensor(), # tensor로 바꾸기
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 정규화 , 수치는 ImageNet에서 널리 사용되는 수치
                                      ])

transforms_val = transforms.Compose([
                                        transforms.Resize([int(300), int(300)], interpolation = InterpolationMode.BILINEAR),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])


train_data_set = datasets.ImageFolder(data_train_path, transform=transforms_train) # data_train_path라는 폴더에서 이미지들을 가져와 위의 증강 과정을 거침
val_data_set = datasets.ImageFolder(data_validation_path, transform=transforms_val)
# dict 지정
dataloaders, batch_num = {}, {}

dataloaders['train'] = DataLoader(train_data_set,
                                    batch_size=hyper_param_batch, # 몇개씩 배치할 것인지
                                    shuffle=True, # epoch마다 섞을 것인지

                                    num_workers=4) # 크면 클수록 데이터를 빠르게 로드할 수 있지만 , 메모리 낭비가 심함
dataloaders['val'] = DataLoader(val_data_set,
                                    batch_size=hyper_param_batch,
                                    shuffle=False,
                                    num_workers=4)

batch_num['train'], batch_num['val'] = len(train_data_set), len(val_data_set) # 학습할 데이터셋들의 크기 확인

print('batch_size : %d,  train/val : %d / %d' % (hyper_param_batch, batch_num['train'], batch_num['val'])) # 최종적으로 학습할 데이터 규모 확인

class_names = train_data_set.classes
print(class_names) # 사용될 4개의 클래스 확인

def train_model(model, criterion, optimizer, scheduler, num_epochs=25, patience=5):
    start_time = time.time()

    since = time.time()
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    consecutive_epochs_without_improvement = 0  # 연속 에포크 동안 개선이 없는 횟수를 세는 카운터

    for epoch in range(num_epochs):
        print('에포크 {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        epoch_start = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            num_cnt = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)

            if phase == 'train':
                scheduler.step()

            epoch_loss = float(running_loss / num_cnt)
            epoch_acc = float((running_corrects.double() / num_cnt).cpu()*100)

            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_loss.append(epoch_loss)
                val_acc.append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

                # 세이브 포인트 저장
                save_checkpoint(epoch, model, optimizer, best_acc)
                consecutive_epochs_without_improvement = 0
                model.load_state_dict(best_model_wts)
                print('==> 최고 성능 모델 저장 - %d / %.1f'%(best_idx, best_acc))
            elif phase == 'train':
                consecutive_epochs_without_improvement += 1

        epoch_end = time.time() - epoch_start
        print('에포크 {} 소요 시간 {:.0f}m {:.0f}s'.format(epoch, epoch_end // 60, epoch_end % 60))
        print()

        if consecutive_epochs_without_improvement >= patience:
            print('조기 종료: 연속된 {}번의 에포크 동안 성능 개선 없음'.format(patience))
            break

    time_elapsed = time.time() - since
    print('학습 완료 소요 시간 {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('최고 성능 Acc: %d - %.1f' %(best_idx, best_acc))

    model.load_state_dict(best_model_wts)

    torch.save(model, PATH + 'aram_'+train_name+'.pt')
    torch.save(model.state_dict(), PATH + 'president_aram_'+train_name+'.pt')


    end_sec = time.time() - start_time
    end_times = str(datetime.timedelta(seconds=end_sec)).split('.')
    end_time = end_times[0]
    print("종료 시간 :", end_time)

    return model, best_idx, best_acc, train_loss, train_acc, val_loss, val_acc


criterion = nn.CrossEntropyLoss() # 손실함수

optimizer_ft = optim.Adam(model.parameters(),lr = 1e-4) # 모델 활성화함수를 Adam , 학습률은 1e - 4로 설정
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) # step_size만큼의 에포크마다 학습률에 gamma를 곱해서 갱신, 0.001

num_epochs = 100
train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)

cuda:0
600
Loaded pretrained weights for efficientnet-b7
batch_size : 32,  train/val : 28873 / 8248
['[원천]비듬_0.양호', '[원천]비듬_1', '[원천]비듬_2.중등도', '[원천]비듬_3.중증']
에포크 0/99
----------
train Loss: 0.1836 Acc: 92.7372
==> 최고 성능 모델 저장 - 0 / 92.7
val Loss: 1.0164 Acc: 73.5451
에포크 0 소요 시간 8m 10s

에포크 1/99
----------
train Loss: 0.1455 Acc: 94.3754
==> 최고 성능 모델 저장 - 1 / 94.4
val Loss: 1.2969 Acc: 72.5388
에포크 1 소요 시간 8m 10s

에포크 2/99
----------
train Loss: 0.1075 Acc: 96.0378
==> 최고 성능 모델 저장 - 2 / 96.0
val Loss: 1.6435 Acc: 71.2173
에포크 2 소요 시간 8m 8s

에포크 3/99
----------
train Loss: 0.0791 Acc: 97.0388
==> 최고 성능 모델 저장 - 3 / 97.0
val Loss: 1.4194 Acc: 71.9690
에포크 3 소요 시간 8m 8s

에포크 4/99
----------
train Loss: 0.0643 Acc: 97.5756
==> 최고 성능 모델 저장 - 4 / 97.6
val Loss: 1.3079 Acc: 73.5451
에포크 4 소요 시간 8m 8s

에포크 5/99
----------
train Loss: 0.0555 Acc: 97.9877
==> 최고 성능 모델 저장 - 5 / 98.0
val Loss: 1.5923 Acc: 72.1508
에포크 5 소요 시간 8m 8s

에포크 6/99
----------
train Loss:

(EfficientNet(
   (_conv_stem): Conv2dStaticSamePadding(
     3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
     (static_padding): ZeroPad2d((0, 1, 0, 1))
   )
   (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
   (_blocks): ModuleList(
     (0): MBConvBlock(
       (_depthwise_conv): Conv2dStaticSamePadding(
         64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
         (static_padding): ZeroPad2d((1, 1, 1, 1))
       )
       (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
       (_se_reduce): Conv2dStaticSamePadding(
         64, 16, kernel_size=(1, 1), stride=(1, 1)
         (static_padding): Identity()
       )
       (_se_expand): Conv2dStaticSamePadding(
         16, 64, kernel_size=(1, 1), stride=(1, 1)
         (static_padding): Identity()
       )
       (_project_conv): Conv2dStaticSamePadding(
         64, 32, kernel_size=(1, 1), strid

In [ ]:
"""
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
"""

In [ ]:
from google.colab import files

# 다운로드할 파일의 경로
file_path = '/content/scalp_weights/aram_model2.pt'

# 파일 다운로드
files.download(file_path)

In [ ]:
import os

# 원하는 코드를 여기에 작성하세요
# 예: print("Hello, World!")

# 런타임 종료
os.kill(os.getpid(), 9)